# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [14]:
# constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

OLLAMA_API = "http://localhost:11434/v1"

openai = OpenAI(api_key = api_key)
ollama_via_openai = OpenAI(base_url=OLLAMA_API, api_key='ollama')

In [22]:
# set up environment
def get_answer(question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": "Give a good desrciptive answer with code examples"},
            {"role": "user", "content": question}
      ],
        stream=True,
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)
    # result = response.choices[0].message.content
    # return json.loads(result)

def get_answer_from_lama(question):
    messages=[
            {"role": "system", "content": "Give a good desrciptive answer with code examples"},
            {"role": "user", "content": question}
      ]
    # payload = {
    #     "model": MODEL_LLAMA,
    #     "messages": messages,
    #     "stream": True
    # }
    response = ollama_via_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=messages,
    )
    print(response.choices[0].message.content)

In [19]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [20]:
# Get gpt-4o-mini to answer, with streaming
get_answer(question)

The code snippet you've provided uses Python's `yield from` statement in conjunction with set comprehension to generate a sequence of authors from a collection of books. Let's break down the components of the code to understand its functionality and purpose.

### Breakdown of the Code

1. **Understanding `books`**:
   - Here, `books` is expected to be an iterable (like a list) containing dictionaries. Each dictionary represents a book and includes various attributes, one of which is "author".

2. **Set Comprehension**: 
   - The expression `{book.get("author") for book in books if book.get("author")}` is a set comprehension. It creates a set containing the authors of the books.
   - `book.get("author")`: This retrieves the value associated with the key "author" from each book dictionary.
   - `if book.get("author")`: This is a filter condition that ensures only books with a valid "author" (i.e., not `None` or empty) are considered. If the author does not exist for that particular book, it is skipped.

3. **Using `yield from`**:
   - The `yield from` statement is used to yield all values from another iterable. It simplifies the process of yielding values from a generator. In this case, it takes each element from the set generated by the set comprehension and yields it individually.
   - This means within a generator function, `yield from` is not just yielding a single author at a time, but rather it’s yielding each author returned from the set.

### Why Use This Code?

1. **Elimination of Duplicates**: 
   - A set automatically handles duplicate values. So if there are multiple books by the same author, they will only appear once in the final output.

2. **Using `yield from`**: 
   - This is a concise and efficient way to yield multiple values without having to write a loop that yields each value one by one.

3. **Handling Missing Authors Gracefully**:
   - By using `if book.get("author")`, the code ensures that no `None` or empty strings are included in the results, which can be particularly helpful for maintaining data quality and integrity.

### Example Code

Here’s a complete example that shows how this code could fit into a generator function:

python
def get_unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Sample book data
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": "Author A"},  # duplicate author
    {"title": "Book 4", "author": None},        # no author
    {"title": "Book 5", "author": "Author C"},
    {"title": "Book 6", "author": ""},          # no author
]

# Usage example
for author in get_unique_authors(books):
    print(author)

# Output will be:
# Author A
# Author B
# Author C


### Summary
- The provided code helps to generate a unique list of authors from a collection of book dictionaries while gracefully handling missing or empty author data.
- It utilizes set comprehension for deduplication and `yield from` for efficient yielding of results in a generator function context.

In [23]:
# Get Llama 3.2 to answer
get_answer_from_lama(question)

**Understanding the Code**

This line of code utilizes a feature called "generator expressions" in Python, which allows you to perform operations on iterables without fully loading them into memory. The specific syntax employed here is `yield from {expression} for iterable`.

Let's break this down:

- **`yield from`**: This keyword is used to delegate the execution of another generator or iterator. It tells Python to "yield" the results of the expression inside, rather than running it itself.

- `{expression}`: This is a dictionary literal that contains an expression (in this case, `book.get("author")`) and potentially other key-value pairs. The keys are specified using square brackets `[...]`.

- `for book in books`: This part is an iteration over another iterable (`books`).

Now, let's put it all together.

**What the Code Does**

The code fetches authors from a collection of books and yields them one by one.

Here's a more verbose equivalent for those not familiar with generator exp